<h1 align=center><font size = 5>Week 3 Assignment: Segmenting and Clustering Neighborhoods in Toronto</font></h1> <p>By Colin Kagame.</p>

### Part 1 : Parsing a Wikepidea page consisting Canada's postal codes and converting the data to a Pandas Dataframe.

##### I chose to analyse borough names that had the word "Toronto" in them, more specifically I later concentrated on exploring Downtown Toronto further in this project.

<u> Importing Libraries and setting up environment. </u>

In [53]:
import pandas as pd # library for data analsysis
import numpy as np # # library to process data as dataframes
pd.set_option('display.max_columns', None) # library for data analsysis
pd.set_option('display.max_rows', None)

import requests # library to handle requests

import urllib.request #cols
from urllib.request import urlopen

import json  # library to handle JSON files

!pip install geopy
import geopy #cols

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# -------------- cols
import matplotlib.pyplot as plt
%matplotlib inline

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML
#--------------- cols

#import k-means from clustering stage #cols
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install beautifulsoup4
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


<u> Parsing HTML </u>

In [54]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')
#-------------------------------------------------------
wikipedia_table = soup.body.table.tbody

<u> Dataframe creation </u>

In [55]:
def get_cell(element):
    cells = element.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

In [56]:
def get_row():    
    data = []  
    
    for tr in wikipedia_table.find_all('tr'):
        row = get_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [57]:
data = get_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<u> Data cleansing </u>

##### Data is sorted by Postcode and borough.

In [58]:
df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['Postcode','Borough'])
#df1 = df1.sort_values(by='Postcode')


df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)

df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [59]:
df_postcodes = df1['Postcode']
df_postcodes.drop_duplicates(inplace=True)
df2 = pd.DataFrame(df_postcodes)
df2['Borough'] = '';
df2['Neighbourhood'] = '';


df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index:
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index:
    s = df2.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df2.iloc[i,2 ] = s
    


In [60]:
    df2.shape

(103, 3)

### Part 2 : Using the Geocoder package/csv file to create the dataframe. I sorted my data by Postcode and Borough.

In [61]:
#import pandas as pd

import csv #cols

dfx = pd.DataFrame(df2) #cols : uncommented as task1.csv already created.
#dfx.to_csv('task1.csv', index=False) # -----> csv
dfx = pd.read_csv('task1.csv') #cols
dfx.head() #cols

# df = pd.read_csv('task1.csv')
# df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [62]:
!pip install geocoder
import geocoder

def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

get_latlng('M4G')

[43.70941386000004, -79.36309957799995]

##### Dataframe consisting coordinates. I have decided to sort the data by postcode and borough.

In [323]:
#1. POSTAL CODES dataframe.
postal_codes = dfx['Postcode']
#postal_codes.head()

data = dfx
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)
#df.head()


#2. COORDINATES dataframe.
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
#df_coords.head()

result = pd.concat([df, df_coords], axis=1).reindex(df.index)
#result.head()

#just coordnates.
#x = df_coords[df.Postcode == 'M5G']
#x.head()

TorontoData = pd.DataFrame(result) #cols : uncommented as task2.csv already created.
#TorontoData.to_csv('task2.csv', index=False) #  -----> csv
TorontoData = pd.read_csv('task2.csv') #consists latitudes and longitudes.
TorontoData.head(10) # shows the scrapped data organised into a dataframe as required.

# To CSV File2? Code above for this.
#dfx.to_csv('task2.csv', index=False) ..dfx is postode,borough & neighborhood only.
#dfx

#.............................above works


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813
5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.726881,-79.265694
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.713340,-79.284942
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.723538,-79.228353
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696448,-79.265642


##### Testing with M5G below.

In [64]:
#coordinates with other data.
x = result[df.Postcode == 'M5G']
x.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653


### Part 3 : Exploring and clustering Toronto neighborhoods.

In [65]:
#had the def get_latlng code here but removed it.

##### Number of boroughs and neighbourhoods in our Toronto dataframe.

In [66]:
print('This dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoData ['Borough'].unique()),
        TorontoData .shape[0]
    )
)

This dataframe has 10 boroughs and 103 neighborhoods.


In [67]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto City are 43.6534817, -79.3839347.


##### Analysing Boroughs of Toronto that contain the word "Toronto"

In [68]:
# Boroughs that contain the word Toronto. 

df1 = TorontoData[TorontoData['Borough'].str.contains("Toronto")] 
#print(df1)
df1

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.678148,-79.295349
41,M4K,East Toronto,"The Danforth West, Riverdale",43.683424,-79.354564
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668291,-79.315578
43,M4M,East Toronto,Studio District,43.648000,-79.339260
44,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415
45,M4P,Central Toronto,Davisville North,43.713171,-79.388870
46,M4R,Central Toronto,North Toronto West,43.714139,-79.406456
47,M4S,Central Toronto,Davisville,43.703327,-79.385649
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.690328,-79.383522
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686378,-79.402372


##### Visualising neighbourhoods in dataframe above on a map.

In [69]:
import folium
# creating map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Foursquare venues.

In [70]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [71]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

##### Dataframe containing sushi restaurants in Downtown Toronto.

In [72]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d1d2941735
neighborhoods = TorontoData[TorontoData['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_venues_sushi = getNearbyVenues(names=neighborhoods['Neighbourhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
toronto_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"St. James Town, Cabbagetown",43.667656,-79.367326,Kibo Sushi,43.665617,-79.368388,Sushi Restaurant
1,"St. James Town, Cabbagetown",43.667656,-79.367326,Kibo Sushi House,43.671375,-79.378597,Sushi Restaurant
2,"St. James Town, Cabbagetown",43.667656,-79.367326,Qi sushi,43.662552,-79.364258,Sushi Restaurant
3,Church and Wellesley,43.666659,-79.381472,Torch Pressed Sushi,43.666394,-79.380994,Sushi Restaurant
4,Church and Wellesley,43.666659,-79.381472,FUSHIMI Authentic Japanese Cuisine,43.665152,-79.380719,Sushi Restaurant


##### Dataframe 'shape'

In [73]:
toronto_venues_sushi.shape

(579, 7)

##### Visualising sushi restuarants in Downtown Toronto.

In [74]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [75]:
map_toronto_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(toronto_venues_sushi, 'red', map_toronto_sushi)

map_toronto_sushi

##### Unique categories in the neighboorhoods of DownTown Toronto.

In [76]:
toronto_grouped = toronto_venues_sushi.groupby('Neighborhood').count()
toronto_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,31,31,31,31,31,31
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,49,49,49,49,49,49
Christie,6,6,6,6,6,6
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
"First Canadian Place, Underground city",50,50,50,50,50,50
"Garden District, Ryerson",42,42,42,42,42,42
"Kensington Market, Chinatown, Grange Park",35,35,35,35,35,35


In [77]:
print('There are {} unique categories.'.format(len(toronto_venues_sushi['Venue Category'].unique())))

There are 9 unique categories.


##### 3. Analyzing Each Neighborhood.

In [78]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_sushi['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Asian Restaurant,Fast Food Restaurant,Grocery Store,Hawaiian Restaurant,Japanese Restaurant,Korean Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant
0,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,1
1,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,1
2,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,1
3,Church and Wellesley,0,0,0,0,0,0,0,0,1
4,Church and Wellesley,0,0,0,0,0,0,0,0,1


##### Grouping by category.

In [79]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Asian Restaurant,Fast Food Restaurant,Grocery Store,Hawaiian Restaurant,Japanese Restaurant,Korean Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant
0,Berczy Park,0.032258,0.000000,0.000000,0.000000,0.129032,0.000000,0.193548,0.000000,0.645161
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.062500,0.062500,0.000000,0.000000,0.062500,0.812500
2,Central Bay Street,0.020408,0.020408,0.020408,0.000000,0.122449,0.000000,0.020408,0.000000,0.795918
3,Christie,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.833333
4,Church and Wellesley,0.033333,0.033333,0.000000,0.000000,0.100000,0.033333,0.033333,0.000000,0.766667
5,"Commerce Court, Victoria Hotel",0.020000,0.000000,0.020000,0.000000,0.120000,0.000000,0.120000,0.000000,0.720000
6,"First Canadian Place, Underground city",0.020000,0.000000,0.020000,0.000000,0.140000,0.000000,0.120000,0.000000,0.700000
7,"Garden District, Ryerson",0.023810,0.023810,0.000000,0.000000,0.119048,0.000000,0.000000,0.000000,0.833333
8,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.028571,0.028571,0.057143,0.000000,0.000000,0.028571,0.857143
9,"Queen's Park, Ontario Provincial Government",0.022222,0.022222,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.844444


##### Top 5 venues.

In [292]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [293]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Sushi Restaurant,Restaurant,Japanese Restaurant,Asian Restaurant,Seafood Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Sushi Restaurant,Seafood Restaurant,Japanese Restaurant,Hawaiian Restaurant,Restaurant
2,Central Bay Street,Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Fast Food Restaurant
3,Christie,Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Restaurant,Korean Restaurant
4,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Korean Restaurant,Fast Food Restaurant


##### Clustering neighbourhoods in DownTown Toronto.

In [294]:
# set number of clusters
kclusters = 5 

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0,).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 3, 3, 3, 4, 4, 3, 0, 3], dtype=int32)

In [295]:
# Add clustering labels 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


#toronto_merged = neighborhoods

toronto_merged = TorontoData[TorontoData['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_merged = (toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood'))
#removing rows with NaN values.
toronto_merged = toronto_merged.dropna()
#converting cluster labels to integers.
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged



,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667656,-79.367326,2,Sushi Restaurant,Seafood Restaurant,Restaurant,Korean Restaurant,Japanese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472,3,Sushi Restaurant,Japanese Restaurant,Restaurant,Korean Restaurant,Fast Food Restaurant
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,2,Sushi Restaurant,Seafood Restaurant,Restaurant,Korean Restaurant,Japanese Restaurant
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,3,Sushi Restaurant,Japanese Restaurant,Fast Food Restaurant,Asian Restaurant,Seafood Restaurant
5,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,4,Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Asian Restaurant
6,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855,1,Sushi Restaurant,Restaurant,Japanese Restaurant,Asian Restaurant,Seafood Restaurant
7,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653,3,Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Fast Food Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650542,-79.384116,4,Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Asian Restaurant
10,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.646923,-79.381626,4,Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Asian Restaurant
11,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648283,-79.378750,4,Sushi Restaurant,Restaurant,Japanese Restaurant,Grocery Store,Asian Restaurant


In [296]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [316]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,"Kensington Market, Chinatown, Grange Park",Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Hawaiian Restaurant,Grocery Store
14,"CN Tower, King and Spadina, Railway Lands, Har...",Sushi Restaurant,Seafood Restaurant,Japanese Restaurant,Hawaiian Restaurant,Restaurant


In [317]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Berczy Park,Sushi Restaurant,Restaurant,Japanese Restaurant,Asian Restaurant,Seafood Restaurant


In [318]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,"St. James Town, Cabbagetown",Sushi Restaurant,Seafood Restaurant,Restaurant,Korean Restaurant,Japanese Restaurant
3,"Regent Park, Harbourfront",Sushi Restaurant,Seafood Restaurant,Restaurant,Korean Restaurant,Japanese Restaurant
12,"University of Toronto, Harbord",Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Restaurant,Korean Restaurant


In [319]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Korean Restaurant,Fast Food Restaurant
4,"Garden District, Ryerson",Sushi Restaurant,Japanese Restaurant,Fast Food Restaurant,Asian Restaurant,Seafood Restaurant
7,Central Bay Street,Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Fast Food Restaurant
17,Christie,Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Restaurant,Korean Restaurant
18,"Queen's Park, Ontario Provincial Government",Sushi Restaurant,Japanese Restaurant,Fast Food Restaurant,Asian Restaurant,Seafood Restaurant


In [321]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,St. James Town,Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Asian Restaurant
8,"Richmond, Adelaide, King",Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Asian Restaurant
10,"Toronto Dominion Centre, Design Exchange",Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Asian Restaurant
11,"Commerce Court, Victoria Hotel",Sushi Restaurant,Restaurant,Japanese Restaurant,Grocery Store,Asian Restaurant
15,Stn A PO Boxes,Sushi Restaurant,Japanese Restaurant,Restaurant,Seafood Restaurant,Grocery Store
16,"First Canadian Place, Underground city",Sushi Restaurant,Japanese Restaurant,Restaurant,Grocery Store,Asian Restaurant
